In [ ]:
!pip install tensorflow


In [128]:
import cv2
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# 參數設定
FRAME_SKIP = 30  # 每隔 30 幀處理一次
MAX_SECONDS = 60  # 處理影片前 60 秒
VIDEO_PATH = 'texture_video.avi'
SAMPLES_PER_SECOND_TRAINING = 5  # 前 24 秒每秒取樣 5 張影像
TEST_DURATION = 60  # 測試集持續時間 (秒)
OUTPUT_PATH = 'C:/Users/User/Desktop/學/大學/IP/IP_final_project/output_video.avi'

# 讀取影片逐幀處理
cap = cv2.VideoCapture(VIDEO_PATH)

# 檢查影片是否成功加載
if not cap.isOpened():
    raise ValueError(f"Cannot open video file at path: {VIDEO_PATH}")

fps = int(cap.get(cv2.CAP_PROP_FPS))  # 取得影片的 FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 影片寬度
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 影片高度
frames = []

frame_count = 0
training_frames = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or (frame_count > fps * (MAX_SECONDS + TEST_DURATION)):
        break
    
    # 前24秒均勻取樣 120 個影像（每秒 5 個）
    if frame_count < fps * 24:
        if frame_count % (fps // SAMPLES_PER_SECOND_TRAINING) == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (64, 64)) / 255.0  # 標準化至 0~1
            frames.append(resized)
            training_frames += 1
    
    frame_count += 1

cap.release()

# 檢查 frames 形狀並擴展維度
frames = np.array(frames)  # (N, 64, 64)
print("Before expanding dimensions:", frames.shape)

# 檢查 frames 是否為空，避免後續出現問題
if frames.size == 0:
    raise ValueError("The frames array is empty. Check video processing.")

# 將 frames 的形狀擴展為 (N, 64, 64, 1)
frames = np.expand_dims(frames, axis=-1)  # (N, 64, 64, 1)
print("After expanding dimensions:", frames.shape)

# 建立自編碼器
input_img = Input(shape=(64, 64, 1))
x = Conv2D(16, 3, activation='relu', padding='same')(input_img)
x = MaxPooling2D(2, padding='same')(x)
x = UpSampling2D(2)(x)
decoded = Conv2D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# 使用正常影像訓練
autoencoder.fit(frames, frames, epochs=20, batch_size=8, shuffle=True)

# 重新開啟影片處理
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0

# 設定輸出影片
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or (frame_count > fps * (MAX_SECONDS + TEST_DURATION)):
        break
    
    if frame_count % FRAME_SKIP == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (64, 64)) / 255.0
        # 重塑影像維度為 (1, 64, 64, 1)，以符合模型輸入格式
        reconstructed = autoencoder.predict(np.expand_dims(resized, axis=[0, -1]))  # 維度變為 (1, 64, 64, 1)
        
        # 計算差異
        diff = np.abs(reconstructed[0].squeeze() - resized) * 255
        
        # 偵測瑕疵區域（差異大於閾值的區域）
        threshold = 40  # 設定閾值
        _, diff_binary = cv2.threshold(diff.astype(np.uint8), threshold, 255, cv2.THRESH_BINARY)
        
        # 找出瑕疵區域輪廓
        contours, _ = cv2.findContours(diff_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # 在原圖上標註瑕疵區域（以紅色標註）
        for contour in contours:
            if cv2.contourArea(contour) > 1:  # 篩選較大的區域作為瑕疵
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)  # 紅色矩形框
                cv2.putText(frame, 'Anomaly', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # 將標註後的影像寫入新影片，設定每幀顯示 0.5 秒
        out.write(frame)

    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Before expanding dimensions: (120, 64, 64)
After expanding dimensions: (120, 64, 64, 1)
Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0085
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0025
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0023 
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020
Epoch 10/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019
Epoch 11/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019
Epoch 12/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019 
Epoch 13/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018
Epoch 14/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.

In [131]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# 設定參數
IMAGE_PATHS = [
    "image1.png",
    "image2.png",
]
OUTPUT_PATHS = [
    "C:/Users/User/Desktop/學/大學/IP/IP_final_project/image1_annotated.png",
    "C:/Users/User/Desktop/學/大學/IP/IP_final_project/image2_annotated.png",
]
THRESHOLD = 14  # 瑕疵偵測閾值
MIN_CONTOUR_AREA =10  # 瑕疵區域最小面積

# 載入訓練好的自編碼器模型
# (確保 autoencoder 模型已在記憶體中，如果需要存檔模型可使用 autoencoder.save(filepath) 先行保存)
autoencoder = autoencoder  # 假設變數 autoencoder 仍然有效

for img_path, output_path in zip(IMAGE_PATHS, OUTPUT_PATHS):
    # 載入影像
    image = cv2.imread(img_path)
    if image is None:
        raise ValueError(f"Cannot load image from path: {img_path}")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 轉灰階
    resized = cv2.resize(gray, (64, 64)) / 255.0  # 標準化並縮放

    # 重建影像
    reconstructed = autoencoder.predict(np.expand_dims(resized, axis=[0, -1]))[0].squeeze()

    # 計算差異
    diff = np.abs(reconstructed - resized) * 255

    # 偵測瑕疵區域
    _, diff_binary = cv2.threshold(diff.astype(np.uint8), THRESHOLD, 255, cv2.THRESH_BINARY)

    # 找出輪廓
    contours, _ = cv2.findContours(diff_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 標註瑕疵區域
    for contour in contours:
        if cv2.contourArea(contour) > MIN_CONTOUR_AREA:  # 篩選較大的瑕疵區域
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x * 8, y * 8), (x * 8 + w * 8, y * 8 + h * 8), (0, 0, 255), 2)
            cv2.putText(image, "Anomaly", (x * 8, y * 8 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # 顯示標註結果
    cv2.imshow(f"Annotated Image - {img_path}", image)

    # 等待按鍵事件後關閉顯示窗口
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # 儲存標註結果
    cv2.imwrite(output_path, image)
    print(f"Processed and saved annotated image to {output_path}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processed and saved annotated image to C:/Users/User/Desktop/學/大學/IP/IP_final_project/image1_annotated.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Processed and saved annotated image to C:/Users/User/Desktop/學/大學/IP/IP_final_project/image2_annotated.png


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 影片路徑
video_path = "texture_video.avi"
cap = cv2.VideoCapture(video_path)

# 設定影片總幀數和幀率
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 訓練集和測試集抽樣參數
train_sample_size = 100  # 訓練集的圖片數量
test_sample_size = 20  # 測試集的圖片數量
test_start_time = 24  # 測試集開始的時間點（秒）
test_end_time = test_start_time + 30  # 測試集結束的時間點（秒）

# 讀取已知有瑕疵的圖片，並檢查是否成功讀取
image1_path = "image1.png"
image2_path = "image2.png"

# 使用cv2.imread來讀取圖片
image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

# 檢查是否成功讀取圖片
if image1 is None:
    print(f"Error: Unable to load image at {image1_path}")
    raise FileNotFoundError(f"Image not found at {image1_path}")

if image2 is None:
    print(f"Error: Unable to load image at {image2_path}")
    raise FileNotFoundError(f"Image not found at {image2_path}")

# 確保模板圖片比當前圖像小，否則進行縮放
def resize_template(image, target_height, target_width):
    return cv2.resize(image, (target_width, target_height))

# 這裡設置一個標準大小來縮放模板（可以根據實際情況調整）
target_height = 224
target_width = 224
image1_resized = resize_template(image1, target_height, target_width)
image2_resized = resize_template(image2, target_height, target_width)

# 提取訓練集（前24秒的影片）
def extract_samples(start_time, end_time, sample_size, video_capture):
    samples = []
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    total_frames = int(fps * (end_time - start_time))
    
    # 均勻抽樣
    for i in range(sample_size):
        frame_num = int(i * total_frames / sample_size + start_time * fps)
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = video_capture.read()
        if ret:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            samples.append(gray_frame)
    
    return samples

# 提取測試集（24秒後30秒範圍內）
cap.set(cv2.CAP_PROP_POS_FRAMES, test_start_time * fps)
test_samples = extract_samples(test_start_time, test_end_time, test_sample_size, cap)

# 基於模板匹配進行瑕疵檢測
def detect_defects(image, sample1, sample2):
    # 使用模板匹配檢測瑕疵
    res1 = cv2.matchTemplate(image, sample1, cv2.TM_CCOEFF_NORMED)
    res2 = cv2.matchTemplate(image, sample2, cv2.TM_CCOEFF_NORMED)
    
    # 設定閾值，過高的匹配度為瑕疵
    threshold = 0.8
    loc1 = np.where(res1 >= threshold)
    loc2 = np.where(res2 >= threshold)
    
    # 標註瑕疵區域
    defects = []
    for pt in zip(*loc1[::-1]):
        defects.append(pt)
    for pt in zip(*loc2[::-1]):
        defects.append(pt)
    
    return defects

# 輪廓檢測
def find_and_draw_contours(defects, image):
    # 創建空白圖片，顯示瑕疵區域
    black_image = np.zeros_like(image)
    for defect in defects:
        cv2.rectangle(black_image, defect, (defect[0] + image1_resized.shape[1], defect[1] + image1_resized.shape[0]), 255, -1)
    
    # 查找輪廓
    contours, _ = cv2.findContours(black_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 顯示瑕疵輪廓
    contour_image = np.zeros_like(image)
    cv2.drawContours(contour_image, contours, -1, (255), 1)
    
    return contour_image

# 測試集瑕疵檢測和輪廓繪製
defects = []
for test_image in test_samples:
    defects.extend(detect_defects(test_image, image1_resized, image2_resized))
    
# 顯示結果
contour_images = []
for defect_image in defects:
    contour_image = find_and_draw_contours(defect_image, test_samples[0])  # 使用第一張測試圖像來顯示輪廓
    contour_images.append(contour_image)

# 顯示輪廓結果
plt.figure(figsize=(10, 10))
for i, contour_image in enumerate(contour_images):
    plt.subplot(4, 5, i + 1)
    plt.imshow(contour_image, cmap='gray')
    plt.axis('off')
plt.show()

# 釋放資源
cap.release()
